In [1]:
import pysolr 

In [2]:
# SOLR
solr = pysolr.Solr('http://localhost:8983/solr/collection/', timeout=10)

In [3]:
id = 'bkc-5'
id

'bkc-5'

In [10]:
def DeleteItemSolr(id):
    r = solr.search(q=f"id:{id}", fl="*,[child]")
    [doc] = r.docs
    itemOf = doc['itemOf']['id']
    instance = itemOf.split("/")[-1]
    hasItem = f"{instance}/hasItem/{id}"
 

    return [itemOf, hasItem, id]

d = DeleteItemSolr('bkc-5')
print(d)

x = solr.delete(id=d, commit=True)
x

['bkc-5/itemOf/bkc-2', 'bkc-2/hasItem/bkc-5', 'bkc-5']


'<?xml version="1.0" encoding="UTF-8"?>\n<response>\n\n<lst name="responseHeader">\n  <int name="status">0</int>\n  <int name="QTime">89</int>\n</lst>\n</response>\n'

In [14]:
d = {"id": "bkc-2",
     "hasItem": {"remove": {"id": "bkc-2/hasItem/bkc-7"} }
     }
solr.add([d], commit=True)

'{\n  "responseHeader":{\n    "status":0,\n    "QTime":287}}\n'

In [13]:
solr.delete(id="bkc-3/itemOf/bkc-2", commit=True)

'<?xml version="1.0" encoding="UTF-8"?>\n<response>\n\n<lst name="responseHeader">\n  <int name="status">0</int>\n  <int name="QTime">33</int>\n</lst>\n</response>\n'

In [10]:
solr.delete(id='bkc-2/hasItem/bkc-6')

'<?xml version="1.0" encoding="UTF-8"?>\n<response>\n\n<lst name="responseHeader">\n  <int name="status">0</int>\n  <int name="QTime">1</int>\n</lst>\n</response>\n'

In [6]:
x = solr.delete(id=['bkc-6', "bkc-2/hasItem/bkc-6", itemOf], commit=True)

x

'<?xml version="1.0" encoding="UTF-8"?>\n<response>\n\n<lst name="responseHeader">\n  <int name="status">0</int>\n  <int name="QTime">65</int>\n</lst>\n</response>\n'